In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [ ]:
%cd ML/Project

/content/drive/MyDrive/Colab Notebooks/ML/Project


In [ ]:
!unzip train_data.zip

Streaming output truncated to the last 5000 lines.
  inflating: train_data/HR/L18_102640_216048_s009.jpg  
  inflating: train_data/HR/L18_112496_217088_s018.jpg  
  inflating: train_data/HR/L18_112648_217096_s014.jpg  
  inflating: train_data/HR/L18_112672_217024_s011.jpg  
  inflating: train_data/HR/L18_102216_216352_s022.jpg  
  inflating: train_data/HR/L18_112480_217112_s001.jpg  
  inflating: train_data/HR/L18_102608_216376_s012.jpg  
  inflating: train_data/HR/L18_102664_215952_s016.jpg  
  inflating: train_data/HR/L18_112552_217040_s004.jpg  
  inflating: train_data/HR/L18_101608_216416_s010.jpg  
  inflating: train_data/HR/L18_112664_217024_s002.jpg  
  inflating: train_data/HR/L18_112472_217104_s011.jpg  
  inflating: train_data/HR/L18_112592_217056_s015.jpg  
  inflating: train_data/HR/L18_112528_217232_s005.jpg  
  inflating: train_data/HR/L18_112688_216992_s019.jpg  
  inflating: train_data/HR/L18_112456_217088_s014.jpg  
  inflating: train_data/HR/L18_112592_217016_s017.jpg

In [ ]:
%cd train_data/
!du -sh

/content/drive/MyDrive/Colab Notebooks/ML/Project/train_data
969M	.


In [ ]:
%cd content

/content


In [1]:
import os
if 'google.colab' in str(get_ipython()) and not os.path.exists('/content/DATSR'):
    !git clone -q https://github.com/caojiezhang/DATSR.git /content/DATSR
    # !pip --quiet install -r /content/master/requirements_colab.txt
    # %cd master/notebooks

In [ ]:
%rm -r DATSR

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from datsr.models.archs.datsr_arch import DATSR

# Grayscale transformation function
def grayscale_transform(image):
    image = image.squeeze(0).cpu().numpy().transpose(1, 2, 0)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return torch.tensor(gray_image).unsqueeze(0).cuda()

# Whitening and Coloring Transform (WCT) function
def wct(content_features, style_features):
    c_mean = content_features.mean(dim=[2, 3], keepdim=True)
    s_mean = style_features.mean(dim=[2, 3], keepdim=True)

    content_centered = content_features - c_mean
    style_centered = style_features - s_mean

    c_cov = content_centered @ content_centered.permute(0, 1, 3, 2)
    s_cov = style_centered @ style_centered.permute(0, 1, 3, 2)

    c_eigval, c_eigvec = torch.symeig(c_cov, eigenvectors=True)
    s_eigval, s_eigvec = torch.symeig(s_cov, eigenvectors=True)

    whitening = c_eigvec @ torch.diag_embed(torch.sqrt(1 / (c_eigval + 1e-5))) @ c_eigvec.permute(0, 1, 3, 2)
    coloring = s_eigvec @ torch.diag_embed(torch.sqrt(s_eigval + 1e-5)) @ s_eigvec.permute(0, 1, 3, 2)

    transformed_features = coloring @ (whitening @ content_centered) + s_mean
    return transformed_features

# Phase Replacement (PR) function
def phase_replacement(content_feature, stylized_feature):
    content_fft = torch.fft.fft2(content_feature, dim=(-2, -1))
    stylized_fft = torch.fft.fft2(stylized_feature, dim=(-2, -1))

    amplitude_content = torch.abs(content_fft)
    phase_content = torch.angle(content_fft)

    amplitude_stylized = torch.abs(stylized_fft)

    result = amplitude_stylized * torch.exp(1j * phase_content)
    return torch.fft.ifft2(result, dim=(-2, -1)).real

# Domain Matching Module
class DomainMatchingSR(nn.Module):
    def __init__(self):
        super(DomainMatchingSR, self).__init__()
        self.encoder = models.vgg19(pretrained=True).features[:21]  # Use VGG19 for feature extraction

    def forward(self, lr_image, ref_image):
        # Step 1: Grayscale Transformation
        lr_gray = grayscale_transform(lr_image)
        ref_gray = grayscale_transform(ref_image)

        # Step 2: Feature Extraction (with VGG19)
        lr_features = self.encoder(lr_gray)
        ref_features = self.encoder(ref_gray)

        # Step 3: Whitening and Coloring Transform
        transformed_features = wct(lr_features, ref_features)

        # Step 4: Phase Replacement
        output = phase_replacement(lr_features, transformed_features)

        return output

# Integrated DATSR with Domain Matching
class IntegratedDATSR(nn.Module):
    def __init__(self):
        super(IntegratedDATSR, self).__init__()
        self.datsr = DATSR()  # Load the existing DATSR model
        self.domain_matching = DomainMatchingSR()

    def forward(self, lr_image, ref_image):
        # Apply Domain Matching Module
        domain_matched_image = self.domain_matching(lr_image, ref_image)

        # Pass through DATSR
        output_sr = self.datsr(domain_matched_image)

        return output_sr

ModuleNotFoundError: No module named 'datsr'

In [ ]:
!cat DATSR/requirements.txt

scipy>=1.11.4
pandas>=2.2.0
category-encoders>=2.6.3
gdown>=5.1.0
GPy>=1.13.1
imbalanced_learn>=0.12.0
ipywidgets>=8.1.1
jupyterlab>=4.1.0
matplotlib>=3.8.2
mlxtend>=0.23.1
openml>=0.14.2
seaborn>=0.13.2
tensorflow_addons>=0.23.0
graphviz>=0.20.1
tensorflow
#fancyimpute>=0.7.0


In [ ]:
!pip --quiet install -r /content/DATSR/requirements_colab.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 6.9 MB/s eta 0:00:00
   ━

In [3]:
# pip install mmcv==0.4.4
# !nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install mmcv==2.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.1/479.1 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.3/452.3 kB 34.1 MB/s eta 0:00:00


In [ ]:
%cd ..

/content


In [ ]:
!python DATSR/datsr/test.py -opt "options/test/test_restoration_mse.yml"

Traceback (most recent call last):
  File "/content/DATSR/datsr/test.py", line 8, in <module>
    from models import create_model
  File "/content/DATSR/datsr/models/__init__.py", line 17, in <module>
    _model_modules = [
  File "/content/DATSR/datsr/models/__init__.py", line 18, in <listcomp>
    importlib.import_module(f'models.{file_name}')
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "/content/DATSR/datsr/models/sr_model.py", line 9, in <module>
    import models.networks as networks
  File "/content/DATSR/datsr/models/networks.py", line 1, in <module>
    from models.archs import _arch_modules
  File "/content/DATSR/datsr/models/archs/__init__.py", line 15, in <module>
    _arch_modules = [
  File "/content/DATSR/datsr/models/archs/__init__.py", line 16, in <listcomp>
    importlib.import_module(f'models.archs.{file_name}')
  File "/usr/lib/python3.10/importlib/__init__.py", 